In [8]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import datetime as dt
import time
import random

In [9]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# JSON 데이터 불러오기
with open("seodaemun_restaurant_200.json", "r", encoding="utf-8") as file:
    restaurant_data = json.load(file)

# 리뷰 데이터를 저장할 리스트
review_results = []

# '더보기' 버튼을 클릭하여 모든 리뷰 로드
def load_all_reviews():
    while True:
        try:
            more_button = driver.find_element(By.XPATH, '//a[contains(text(), "더보기")]')
            more_button.click()
            time.sleep(0.5)  # 클릭 후 로딩 시간
        except Exception:
            break  # 더보기 버튼이 없으면 종료

# 리뷰 데이터 크롤링
for restaurant in restaurant_data:
    url = restaurant["URL"]
    print(f"크롤링 중: {restaurant['가게 이름']} - {url}")
    try:
        driver.get(url)
        time.sleep(3)  # 페이지 로드 대기
        
        # 페이지 스크롤로 모든 리뷰 로드
        load_all_reviews()

        # HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")
        reviews = soup.select("li.pui__X35jYm.EjjAW")  # 리뷰 요소 선택자
        
        for review in reviews:
            # 닉네임
            nickname = review.select_one("div.pui__JiVbY3 > span.pui__uslU0d")
            nickname = nickname.text.strip() if nickname else "N/A"

            # 리뷰 내용
            content = review.select_one("div.pui__vn15t2 > a.pui__xtsQN-")
            content = content.text.strip() if content else "N/A"

            # 작성일
            date = review.select_one("div.pui__QKE5Pr > span.pui__gfuUIT > time")
            date = date.text.strip() if date else "N/A"

            # 재방문 횟수
            revisit = review.select_one("div.pui__QKE5Pr > span.pui__gfuUIT:nth-of-type(2)")
            revisit = revisit.text.strip() if revisit else "N/A"

            # 데이터 저장
            review_results.append({
                "가게 이름": restaurant["가게 이름"],
                "닉네임": nickname,
                "리뷰 내용": content,
                "작성일": date,
                "재방문 여부": revisit
            })
        print(f"{restaurant['가게 이름']} - {len(reviews)}개의 리뷰 수집 완료")

    except Exception as e:
        print(f"에러 발생: {e}")

# 결과를 JSON 파일로 저장
timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"naver_reviews_{timestamp}.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(review_results, file, ensure_ascii=False, indent=4)

print(f"리뷰 크롤링 완료. 결과는 {output_file}에 저장되었습니다.")
driver.quit()

크롤링 중: 역전포장마차 신촌점 - https://pcmap.place.naver.com/restaurant/1984155649/review/visitor#
역전포장마차 신촌점 - 10개의 리뷰 수집 완료
크롤링 중: 헤일우드 - https://pcmap.place.naver.com/restaurant/1095540561/review/visitor#
헤일우드 - 10개의 리뷰 수집 완료
크롤링 중: 정육면체 신촌점 - https://pcmap.place.naver.com/restaurant/1229208931/review/visitor#
정육면체 신촌점 - 10개의 리뷰 수집 완료
크롤링 중: 봉일천 장군집 - https://pcmap.place.naver.com/restaurant/13435774/review/visitor#
봉일천 장군집 - 10개의 리뷰 수집 완료
크롤링 중: 머노까머나 신촌점 - https://pcmap.place.naver.com/restaurant/12791076/review/visitor#
머노까머나 신촌점 - 10개의 리뷰 수집 완료
크롤링 중: 스웨이커피스테이션 - https://pcmap.place.naver.com/restaurant/1826959498/review/visitor#
스웨이커피스테이션 - 10개의 리뷰 수집 완료
크롤링 중: 작은스페인 - https://pcmap.place.naver.com/restaurant/38738511/review/visitor#
작은스페인 - 10개의 리뷰 수집 완료
크롤링 중: 이석덕생면파스타 신촌점 - https://pcmap.place.naver.com/restaurant/1363106336/review/visitor#
이석덕생면파스타 신촌점 - 10개의 리뷰 수집 완료
크롤링 중: 한사발포차 신촌점 - https://pcmap.place.naver.com/restaurant/1337888553/review/visitor#
한사발포차 신촌점 - 10개의 리뷰 수집 완료
크롤링 중